In [23]:
import time
import pprint

import numpy
import pandas
import plotly.express as pltxpr

import stable_baselines3
import gymnasium as gym

import axis_and_allies.run_simulation as run_simulation
import axis_and_allies.reinforcement_learning.super_simple_env as super_simple_env

import pickle

In [28]:
output_template = "env_data_r{}x{}.txt"

In [2]:
IPC_limit = 30
ipc_cost_arr = numpy.array([3,4,6,10,12])

unit_dict = run_simulation.load_units("../../unit_data.json")
pprint.pprint(unit_dict)

{'aircraft carrier': name:  aircraft carrier
id:  9
ipc:  14
attack:  1
temp_attack:  1
defense:  2
unit_type:  naval
move:  2
max_hit_points:  1
cur_hit_points:  1,
 'anti-aircraft artillery': name:  anti-aircraft artillery
id:  3
ipc:  5
attack:  0
temp_attack:  0
defense:  1
unit_type:  land
move:  1
max_hit_points:  1
cur_hit_points:  1,
 'artillery': name:  artillery
id:  1
ipc:  4
attack:  2
temp_attack:  2
defense:  2
unit_type:  land
move:  1
max_hit_points:  1
cur_hit_points:  1,
 'battleship': name:  battleship
id:  10
ipc:  20
attack:  4
temp_attack:  4
defense:  4
unit_type:  naval
move:  2
max_hit_points:  2
cur_hit_points:  2,
 'bomber': name:  bomber
id:  5
ipc:  12
attack:  4
temp_attack:  4
defense:  1
unit_type:  air
move:  6
max_hit_points:  1
cur_hit_points:  1,
 'cruiser': name:  cruiser
id:  8
ipc:  12
attack:  3
temp_attack:  3
defense:  3
unit_type:  naval
move:  2
max_hit_points:  1
cur_hit_points:  1,
 'destroyer': name:  destroyer
id:  7
ipc:  8
attack:  2
te

In [3]:
env = super_simple_env.SuperSimpleEnv(IPC_limit, ipc_cost_arr, unit_dict)
env

/Users/lahr/miniconda3/envs/axis_and_allies/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


IPC_limit:  30
log_ratio_limit:  5.0
action_space:  Box([ 0. -5. -5. -5. -5.], [1. 5. 5. 5. 5.], (5,), float32)
observation_space:  Box(0, 45, (1,), int32)
current_action:  [1. 0. 0. 0. 0.]

In [4]:
model = stable_baselines3.PPO(stable_baselines3.ppo.MlpPolicy, env, verbose=0) # , n_steps=2
env.model = model

In [5]:
start_time = time.time()
model.learn(total_timesteps=1)
end_time = time.time()
duration = end_time - start_time
duration_min = duration / 60.
print("duraton:  {}  duration_min:  {}".format(duration, duration_min))

progress:  100
progress:  200
progress:  300
progress:  400
progress:  500
progress:  600
progress:  700
progress:  800
progress:  900
progress:  1000
progress:  1100
progress:  1200
progress:  1300
progress:  1400
progress:  1500
progress:  1600
progress:  1700
progress:  1800
progress:  1900
progress:  2000
duraton:  12.688237190246582  duration_min:  0.21147061983744303


In [30]:
a = numpy.vstack(env.all_actions)
# a.shape
a_df = pandas.DataFrame(a, columns=["action_{}".format(i) for i in range(a.shape[1])])
# a_df
b_df = pandas.DataFrame(
    numpy.vstack(env.all_opponent_actions),
    columns=["opp_action_{}".format(i) for i in range(len(env.all_opponent_actions[0]))]
)
# b_df
data_df = pandas.concat([a_df, b_df], axis=1)
data_df["result"] = env.all_results
data_df.index.name = "training_cycle"
data_df

,action_0,action_1,action_2,action_3,action_4,opp_action_0,opp_action_1,opp_action_2,opp_action_3,opp_action_4,result
training_cycle,,,,,,,,,,,
0,0.000000,-0.726898,2.244221,-0.142794,-0.138442,0.890887,-0.970383,0.291814,0.624429,-0.961032,-1.000000
1,0.000000,-0.992574,0.005197,0.567344,0.637002,0.000000,-0.931366,-0.294516,3.028534,-1.186259,0.000000
2,0.000000,-1.156900,-0.193230,0.753003,1.349013,0.000000,0.538889,-0.888879,0.146302,1.238795,0.000000
3,1.000000,-1.251098,0.135019,-0.343283,-1.097692,0.795471,-0.319605,-1.702049,1.142350,-0.432597,-0.833333
4,0.000000,0.622274,0.060620,0.762191,-1.515538,0.000000,-0.712765,0.121787,0.507263,-0.825855,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2043,1.000000,0.121700,1.936080,0.356472,-0.501209,0.000000,0.775387,0.396173,-0.662176,0.457125,1.000000
2044,0.000000,-0.522945,0.679708,0.129692,0.489787,0.000000,-0.091615,1.290665,-0.647395,-1.714855,0.000000
2045,0.440993,-0.657565,0.012124,-0.680173,-0.329286,1.000000,2.131335,0.892355,1.054550,0.178677,0.461538


In [31]:
output_file = output_template.format(data_df.shape[0], data_df.shape[1])
print(output_file)

env_data_r2048x11.txt


In [32]:
data_df.to_csv(output_file, sep="\t")